In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

img=cv2.imread('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train/cat.2005.jpg')

pixels=[]

for i in range(img.shape[0]):
    for j in range(img.shape[1]):
        pixels.append(img[i,j])

plt.imshow(img)
plt.show()

In [ ]:
# data loading

import torch 

# 0: cat ; 1: dog

n=500

X=torch.zeros(2*n,3,224,224)
t=torch.zeros(2*n)

cat_indices=np.random.permutation(range(1,n+1))
dog_indices=np.random.permutation(range(1+n+1))

for i in range(n):
    t[i]=0
    
    img=cv2.imread('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train/cat.'+str(cat_indices[i])+'.jpg')
    img=cv2.resize(img,dsize=(224,224))
    
    img=np.transpose(img,(2,0,1))
    
    img=torch.Tensor(img/255)
    
    X[i]=img
    
    if (i+1)%(n/10)==0:
        print('Now processing ',i,'-th image')
        
for i in range(n):
    t[i+n]=1
    
    img=cv2.imread('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train/dog.'+str(dog_indices[i])+'.jpg')
    img=cv2.resize(img,dsize=(224,224))
    
    img=np.transpose(img,(2,0,1))

    img=torch.Tensor(img/255)
    
    X[i+n]=img
    
    if (i+1)%(n/10)==0:
        print('Now processing ',i,'-th image')    

In [ ]:
t=t.type(torch.LongTensor)
print(X.size())
print(t.size())

In [ ]:
import torch.nn as nn
import torchvision.models as models

net=models.alexnet(num_classes=2)

import torch.optim as optim

criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.01,momentum=0.9)

In [ ]:
net=net.train()

mini_batch_size=25
for epoch in range(1):
    
    for i in range(int(n/mini_batch_size)):
        
        X_mini=torch.zeros(2*mini_batch_size,3,224,224)
        t_mini=torch.zeros(2*mini_batch_size)
        
        X_mini[0:mini_batch_size]=X[mini_batch_size*i:(mini_batch_size*i+mini_batch_size)]
        t_mini[0:mini_batch_size]=t[mini_batch_size*i:(mini_batch_size*i+mini_batch_size)]
        
        X_mini[mini_batch_size:2*mini_batch_size]=X[n+mini_batch_size*i:n+(mini_batch_size*i+mini_batch_size)]
        t_mini[mini_batch_size:2*mini_batch_size]=t[n+mini_batch_size*i:n+(mini_batch_size*i+mini_batch_size)]
    
        t_mini=t_mini.type(torch.LongTensor)
    
        output=net(X_mini)
        loss=criterion(output,t_mini)
    
        optimizer.zero_grad()
    
        loss.backward()
    
        if i%10==0:
            print(loss)
    
        optimizer.step()

In [ ]:
# creating output file & loading test data 
n_test=125
ID=range(1,125+1)
Output=pd.DataFrame(index=ID)
Output.index.name='id'

X_test=torch.zeros(n_test,3,224,224)
for i in ID:
    img=cv2.imread('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test/'+str(i)+'.jpg')
    img=cv2.resize(img,dsize=(224,224))
    
    img=np.transpose(img,(2,0,1))
    
    img=torch.Tensor(img/255)
    
    X_test[i-1]=img
    
    if i%1000==0:
        print('Now processing ',i,'-th image')
    


In [ ]:
prediction=[]
mini_batch_size=100
for i in range(int(n_test/mini_batch_size)):
    print('Now processing ',i*100)
    with torch.no_grad():
        net_outputs=net(X_test[i*mini_batch_size:(i+1)*mini_batch_size])
        probs=torch.softmax(net_outputs,dim=1)
        prediction.extend(list(probs.numpy()[:,1]))

In [ ]:
Output['label']=prediction

In [ ]:
Output[0:10]
Output.to_csv('MyOutput.csv')